In [4]:
import tensorflow as tf
from tensorflow.python.ops.rnn import _transpose_batch_time

from datetime import datetime
import numpy as np
import pandas as pd
import glob, os, sys, math, warnings, copy, time
import matplotlib.pyplot as  plt
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# customized ftns 
from preprocessing import *
from features import *
from utilities import *
from model import *
from train import train_all_single_policies
# ---------------------------------------------------------
%matplotlib inline
%load_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
# ---------------------------------------------------------
# directories
main_dir = '../'
game_dir = main_dir+'data/'
Data = LoadData(main_dir, game_dir)
models_path = './models/'

#### Load raw data

In [5]:
# %%time
game_id = '0021500196'
game_data = Data.load_game(game_id)
events_df = pd.DataFrame(game_data['events'])
print('raw events shape:', events_df.shape)
events_df.head(3)

raw events shape: (234, 8)


,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,716.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...",[0],"Empty DataFrame Columns: [GAME_ID, EVENTNUM, E...",1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,701.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...","[1, 2]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,682.36,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224825036, 707.33, 11.37, None, [[-1,...","[3, 4]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,707.33,"{'abbreviation': 'TOR', 'players': [{'playerid..."


#### Get some suplementary data

In [6]:
# # play id to play roles/positions
# id_role = id_position(events_df)
# check_game_roles_duplicates(id_role)

# # its possible that F has similar role as G-f or F-G, we create empty slots to ensure meta order
# # ddentify defending and offending runs (this is included in process_moments)
# court_index = Data.load_csv('./meta_data/court_index.csv')
# court_index = dict(zip(court_index.game_id, court_index.court_position))

# # home and visitor ids
# homeid = events_df.loc[0].home['teamid']
# awayid = events_df.loc[0].visitor['teamid']

### FILTER EVENTS

In [7]:
# events_df.loc[3].playbyplay.to_dict('list')

In [8]:
# events_df.moments[3]

In [9]:
# n_event = 233
# P = PlotGame('0021500196', main_dir, game_dir)
# for i in range(len(events_df.moments[n_event])):
#     P.load_moment2img(game_data, n_event, i)

### as we saw that the playbyplay description of events is not accurate, so for now at least we will not try to filter by events

In [37]:
def remove_non_eleven(events_df, event_length_th=25, verbose=False):
    df = events_df.copy()
    def remove_non_eleven_(moments, event_length_th=25, verbose=False):
        ''' Go through each moment, when encounters balls not present on court,
            or less than 10 players, discard these moments and then chunk the following moments 
            to as another event.

            Motivations: balls out of bound or throwing the ball at side line will
                probably create a lot noise for the defend trajectory learning model.
                We could add the case where players are less than 10 (it could happen),
                but this is not allowed in the model and it requres certain input dimension.

            moments: A list of moments
            event_length_th: The minimum length of an event

            segments: A list of events (or, list of moments) e.g. [ms1, ms2] where msi = [m1, m2]
        '''

        segments = []
        segment = []
        # looping through each moment
        for i in range(len(moments)):
            # get moment dimension
            moment_dim = len(moments[i][5])
            # 1 bball + 10 players
            if moment_dim == 11:
                segment.append(moments[i])
            # less than ten players or basketball is not on the court
            else:
    #             print('less than 11')
                # only grab these satisfy the length threshold
                if len(segment) >= event_length_th:
                    segments.append(segment)
                # reset the segment to empty list
                segment = []
        # grab the last one
        if len(segment) >= event_length_th:
            segments.append(segment)
        if len(segments) == 0:
            if verbose: print('Warning: Zero length event returned')
        return segments
    # process for each event (row)
    df['chunked_moments'] = df.moments.apply(lambda m: remove_non_eleven_(m, event_length_th, verbose))
    # in case there's zero length event
    df = df[df['chunked_moments'].apply(lambda e: len(e)) != 0]
    df['chunked_moments'] = df['chunked_moments'].apply(lambda e: e[0])
    return df['chunked_moments'].values

In [39]:
r = remove_non_eleven(events_df)

In [50]:
type(r[0]) == list

True

In [51]:
events_df = pd.DataFrame({'moments': r})

In [ ]:
def chunk_shotclock(events_df, event_length_th=25, verbose=False):
    df = events_df.copy()
    def chunk_shotclock_(moments, event_length_th, verbose):
        ''' When encounters ~24secs or game stops, chunk the moment to another event.
            shot clock test:
            1) c = [20.1, 20, 19, None,18, 12, 9, 7, 23.59, 23.59, 24, 12, 10, None, None, 10]
              result = [[20.1, 20, 19], [18, 12, 9, 7], [23.59], [23.59], [24, 12, 10]]
            2) c = [20.1, 20, 19, None, None,18, 12, 9, 7, 7, 7, 23.59, 23.59, 24, 12, 10, None, None, 10]
              result = [[20.1, 20, 19], [18, 12, 9, 7], [7], [7], [23.59], [23.59], [24, 12, 10]]

            Motivations: game flow would make sharp change when there's 24s or 
            something happened on the court s.t. the shot clock is stopped, thus discard
            these special moments and remake the following valid moments to be next event.

            moments: A list of moments
            event_length_th: The minimum length of an event
            verbose: print out exceptions or not

            segments: A list of events (or, list of moments) e.g. [ms1, ms2] where msi = [m1, m2] 
        '''

        segments = []
        segment = []
        # naturally we won't get the last moment, but it should be okay
        for i in range(len(moments)-1):
            current_shot_clock_i = moments[i][3]
            next_shot_clock_i = moments[i+1][3]
            # sometimes the shot clock value is None, thus cannot compare
            try:
                # if the game is still going i.e. sc is decreasing
                if next_shot_clock_i < current_shot_clock_i:
                    segment.append(moments[i])
                # for any reason the game is sstopped or reset
                else:
                    # not forget the last moment before game reset or stopped
                    if current_shot_clock_i < 24.:
                        segment.append(moments[i])
                    # add length condition
                    if len(segment) >= event_length_th:
                        segments.append(segment)
                    # reset the segment to empty list
                    segment = []
            # None value
            except Exception as e:
                if verbose: print(e)
                # not forget the last valid moment before None value
                if current_shot_clock_i != None:
                    segment.append(moments[i])    
                if len(segment) >= event_length_th:
                    segments.append(segment)
                # reset the segment to empty list
                segment = []

        # grab the last one
        if len(segment) >= event_length_th:
            segments.append(segment)            
        if len(segments) == 0:
            if verbose: print('Warning: Zero length event returned')
        return segments
    
    # process for each event (row)
    df['chunked_moments'] = df.moments.apply(lambda m: chunk_shotclock_(m, event_length_th, verbose))
    # in case there's zero length event
    df = df[df['chunked_moments'].apply(lambda e: len(e)) != 0]
    df['chunked_moments'] = df['chunked_moments'].apply(lambda e: e[0])
    return df['chunked_moments'].values


In [54]:
r = chunk_shotclock(events_df)
events_df = pd.DataFrame({'moments': r})

In [ ]:
def chunk_halfcourt(events_df, event_length_th=25, verbose=False):
    df = events_df.copy()
    def chunk_halfcourt_(moments, event_length_th, verbose):
        ''' Discard any plays that are not single sided. When the play switches 
            court withhin one event, we chunk it to be as another event
        '''

        # NBA court size 94 by 50 feet
        half_court = 94/2. # feet
        cleaned = []

        # remove any moments where two teams are not playing at either side of the court
        for i in moments:
            # the x coordinates is on the 3rd or 2 ind of the matrix,
            # the first and second is team_id and player_id
            team1x = np.array(i[5])[1:6, :][:, 2]    # player data starts from 1, 0 ind is bball
            team2x = np.array(i[5])[6:, :][:, 2]
            # if both team are on the left court:
            if sum(team1x <= half_court)==5 and sum(team2x <= half_court)==5:
                cleaned.append(i)
            elif sum(team1x >= half_court)==5 and sum(team2x >= half_court)==5:
                cleaned.append(i)

        # if teamns playing court changed during same list of moments,
        # chunk it to another event
        segments = []
        segment = []
        for i in range(len(cleaned)-1):
            current_mean = np.mean(np.array(cleaned[i][5])[:, 2], axis=0)
            current_pos = 'R' if current_mean >= half_court else 'L'
            next_mean = np.mean(np.array(cleaned[i+1][5])[:, 2], axis=0)
            next_pos = 'R' if next_mean >= half_court else 'L'

            # the next moment both team are still on same side as current
            if next_pos == current_pos:
                segment.append(cleaned[i])
            else:
                if len(segment) >= event_length_th:
                    segments.append(segment)
                segment = []
        # grab the last one
        if len(segment) >= event_length_th:
            segments.append(segment)            
        if len(segments) == 0:
            if verbose: print('Warning: Zero length event returned')
        return segments
    
    # process for each event (row)
    df['chunked_moments'] = df.moments.apply(lambda m: chunk_halfcourt_(m, event_length_th, verbose))
    # in case there's zero length event
    df = df[df['chunked_moments'].apply(lambda e: len(e)) != 0]
    df['chunked_moments'] = df['chunked_moments'].apply(lambda e: e[0])
    return df['chunked_moments'].values


In [55]:
r = chunk_shotclock(events_df)
events_df = pd.DataFrame({'moments': r})

In [ ]:
len(single_game2)

In [ ]:
events_df.shape[0]

In [ ]:
# court_index

In [ ]:
court_index = Data.load_csv('./meta_data/court_index.csv')
court_index = dict(zip(court_index.game_id, court_index.court_position))
court_index[int('0021500196')]

In [ ]:
home_id = events_df.loc[0].home['teamid']
home_id

In [ ]:
single_game2[-1][0][5]

In [ ]:
def reorder_teams(input_moments, game_id):
    ''' 1) the matrix always lays as home top and away bot VERIFIED
        2) the court index indicate which side the top team (home team) defends VERIFIED
        
        Reorder the team position s.t. the defending team is always the first 
        
        input_moments: A list moments
        game_id: str of the game id
    '''
    # now we want to reorder the team position based on meta data
    court_index = Data.load_csv('./meta_data/court_index.csv')
    court_index = dict(zip(court_index.game_id, court_index.court_position))
    
    half_court = 94/2. # feet
    home_defense = court_index[int(game_id)]
    moments = copy.deepcopy(input_moments)
    for i in range(len(moments)):
        home_moment_x = np.array(moments[i][5])[1:6,2]
        away_moment_x = np.array(moments[i][5])[6:11,2]
        quarter = moments[i][0]
        # if the home team's basket is on the left
        if home_defense == 0:
            # first half game
            if quarter <= 2:
                # if the home team is over half court, this means they are doing offense
                # and the away team is defending, so switch the away team to top
                if sum(home_moment_x>=half_court) and sum(away_moment_x>=half_court):
                    moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
            # second half game      
            if quarter > 2: # second half game, 3,4 quarter
                # now the home actually gets switch to the other court
                if sum(home_moment_x<=half_court) and sum(away_moment_x<=half_court):
                    moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
        # if the home team's basket is on the right
        elif home_defense == 1:
            # first half game
            if quarter <= 2:
                # if the home team is over half court, this means they are doing offense
                # and the away team is defending, so switch the away team to top
                if sum(home_moment_x<=half_court) and sum(away_moment_x<=half_court):
                    moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
            # second half game      
            if quarter > 2: # second half game, 3,4 quarter
                # now the home actually gets switch to the other court
                if sum(home_moment_x>=half_court) and sum(away_moment_x>=half_court):
                    moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
    return moments

In [ ]:
single_game3 = [reorder_teams(i, game_id) for i in single_game2]

In [ ]:
single_game3[0]